# Incident Summarization

Critical cybersecurity incidents often result in lengthy, complex documentation that is difficult to digest, slowing triage, handoffs, and leadership visibility. The goal is to automatically condense these incident reports into concise summaries tailored to analysts or executives — enabling faster response, clearer communication, and more effective decision-making.

**This notebook addresses a simple use case of incident summarization with longer logs. For end-to-end incident investigation, see the Incident Investigation notebook.**

## Model used for this use case
Both Instruct Model and Reasoning Model can be used, but Instruct Model works well via SageMaker endpoint as no complex reasoning is involved.

**Note**: Update the configuration variables below to match your deployment.

## Configuration
Update these variables to match your SageMaker deployment:

In [ ]:
# UPDATE THESE VARIABLES TO MATCH YOUR DEPLOYMENT
endpoint_name = 'foundation-sec-8b-endpoint'  # Your SageMaker endpoint name
aws_region = 'us-east-1'  # Your AWS region

print(f"Configuration:")
print(f"Endpoint: {endpoint_name}")
print(f"Region: {aws_region}")

## Setup
The setup uses SageMaker endpoint instead of loading the model locally.

In [ ]:
import boto3
import json
import re
from IPython.display import display, Markdown

# Initialize SageMaker runtime client
sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=aws_region)

print(f"Connected to SageMaker endpoint: {endpoint_name}")

In [ ]:
# Generation arguments for summarization tasks
generation_args = {
    "max_new_tokens": 1024,
    "temperature": None,  # Deterministic for consistent summaries
    "repetition_penalty": 1.2,
    "do_sample": False,
    "use_cache": True,
}

print("Generation configuration:")
for key, value in generation_args.items():
    print(f"  {key}: {value}")

In [ ]:
def inference(prompt, system_prompt):
    """Inference function using SageMaker endpoint for incident summarization"""
    
    # Format the conversation with system and user prompts
    formatted_prompt = f"System: {system_prompt}\n\nUser: {prompt}\n\nAssistant: "
    
    # Prepare payload for SageMaker endpoint
    payload = {
        "inputs": formatted_prompt,
        "parameters": generation_args
    }
    
    try:
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Body=json.dumps(payload)
        )
        
        result = json.loads(response['Body'].read().decode())
        
        # Handle different TGI response formats
        if isinstance(result, list) and len(result) > 0:
            generated_text = result[0].get('generated_text', '')
        elif isinstance(result, dict):
            generated_text = result.get('generated_text', str(result))
        else:
            generated_text = str(result)
        
        # Clean up the response (remove the original prompt if it's included)
        if generated_text.startswith(formatted_prompt):
            response_text = generated_text[len(formatted_prompt):].strip()
        else:
            response_text = generated_text.strip()
            
        # Remove any trailing special tokens
        response_text = re.sub(r'<\|.*?\|>$', '', response_text).strip()
        
        return response_text
        
    except Exception as e:
        print(f"Error invoking endpoint: {str(e)}")
        return f"Error: {str(e)}"

# Test the inference function
test_response = inference("Test summarization capability", "You are a helpful assistant.")
print("Test Response:")
print(test_response)

## Incident Summarization Functions

In [ ]:
def make_prompt(incident_document: str, summary_type: str = "executive") -> str:
    """Generate prompts for different types of incident summaries"""
    
    assert summary_type in ["executive", "tactical"], "Invalid summary_type. Use 'executive' or 'tactical'."
    
    if summary_type == "executive":
        instructions = (
            "Your task is to provide a clear, non-technical summary for security leadership.\n"
            "- Focus on what happened, when it happened, and why it matters\n"
            "- Keep it under 5 sentences\n"
            "- Use clear, plain language\n"
            "- Emphasize business impact and remediation status"
        )
    elif summary_type == "tactical":
        instructions = (
            "Your task is to provide a concise bullet-point summary for L2/L3 analysts.\n"
            "- List key events and techniques\n"
            "- Include MITRE ATT&CK techniques where applicable\n"
            "- Use technical language where appropriate\n"
            "- Focus on IOCs, TTPs, and response actions"
        )

    return f"""
    {instructions}
    
    ## INCIDENT DOCUMENTATION
    {incident_document}

    Respond with a clear summary.
    """

print("Incident summarization functions ready!")

## Sample Long Incident Report

Let's analyze a comprehensive incident report and generate different types of summaries.

In [ ]:
long_incident_doc = """
Incident Report: IR-2025-0173

Executive Summary:
On April 4, 2025, the security operations center (SOC) was alerted to anomalous behavior originating from an engineering jump box (ENG-JB01). The incident spanned approximately 18 hours and resulted in the compromise of multiple internal systems, theft of sensitive source code repositories, and signs of attempted exfiltration to a known threat infrastructure host associated with APT29. This report outlines the timeline, impact, attacker techniques, and remediation actions taken.

Timeline of Events:
- 2025-04-04 01:14 UTC: Alert triggered for unusual lateral movement attempt originating from ENG-JB01. Logged by Cisco XDR as \"Possible Pass-the-Hash movement attempt.\"
- 2025-04-04 01:18 UTC: Multiple failed Kerberos authentication attempts on host HR-LAPTOP-5 from ENG-JB01.
- 2025-04-04 01:23 UTC: Successful logon to FIN-SQL-02 using a privileged service account (svc-backup-prod) outside of usual login times.
- 2025-04-04 01:25 UTC: PowerShell execution detected from FIN-SQL-02 running a Base64-encoded command fetching content from hxxp://185.44.76.100/payload.ps1.
- 2025-04-04 01:30 UTC: Scheduled task created named \"WindowsHealth\" on FIN-SQL-02 for persistence.
- 2025-04-04 01:33 UTC: Internal network scanning activity detected from FIN-SQL-02 targeting subnet 10.0.42.0/24.
- 2025-04-04 02:01 UTC: SMB and WMI-based authentication attempts observed toward code repository server (ENG-GIT-SRV).
- 2025-04-04 02:11 UTC: Access to sensitive Git repositories verified using server audit logs.
- 2025-04-04 02:15–04:30 UTC: Multiple 50–150MB file transfers from ENG-GIT-SRV to ENG-JB01. Total volume approx. 3.1GB.
- 2025-04-04 04:45 UTC: Outbound connection to 185.44.76.100 established from ENG-JB01 on port 443, observed via egress firewall logs.
- 2025-04-04 04:52 UTC: Data transfer spike of ~3.2GB to external host.
- 2025-04-04 05:02 UTC: DLP system flagged unauthorized data exfiltration attempt. Incident escalated to IR team.
- 2025-04-04 05:13 UTC: Isolation actions initiated on ENG-JB01, FIN-SQL-02, and ENG-GIT-SRV.
- 2025-04-04 05:55 UTC: Memory capture and forensic triage initiated on all impacted systems.

Tactics and Techniques:
- Initial Access: Credential stuffing or lateral movement via Jump Box (ENG-JB01)
- Credential Access: Possible Pass-the-Hash (T1550.002), credential dumping via LSASS access
- Execution: Obfuscated PowerShell scripts (T1059.001)
- Persistence: Scheduled Task creation (T1053.005)
- Discovery: Network scanning (T1046)
- Collection: Staging of large volumes of code repositories
- Exfiltration: HTTPS outbound to threat infrastructure (T1041)

Impacted Assets:
- ENG-JB01 (Jump Box, staging point)
- FIN-SQL-02 (Finance DB server, used for lateral movement)
- ENG-GIT-SRV (Code repository server, source of exfiltrated data)

Impacted Data:
- At least 12 private Git repositories accessed
- Two contained proprietary ML models and customer integration code
- No indication of PII or PCI data exposure

Root Cause:
The attacker appears to have compromised or reused valid credentials on ENG-JB01, which had elevated network permissions. The exact credential acquisition method is undetermined but likely due to reuse or prior phishing compromise. MFA was not enabled on the jump box account.

Remediation Actions:
- Immediate network isolation of all affected systems
- Reset of all privileged and domain service accounts
- Implementation of LAPS and forced password rotation for local accounts
- Blocking outbound traffic to 185.44.76.100 at the firewall
- Deployment of EDR response rules for persistence techniques
- Initiated rollout of MFA to all bastion and admin systems
- Engaged law enforcement and threat intelligence partners for IOCs

Lessons Learned:
- MFA gaps on high-privilege infrastructure allowed credential reuse
- Scheduled task creation alerts were not being monitored
- EDR rules for base64-encoded PowerShell were disabled in engineering OU
- Repository audit logging was insufficient to detect staging early

Recommendations:
- Mandatory MFA on all privileged and remote-access accounts
- Enable audit logging on all code and data staging systems
- Expand scheduled task alerting across all domains
- Conduct purple team simulation to test lateral movement detection

This report has been reviewed and approved by the incident response lead and executive security team. Further updates will be shared in postmortem review.

-- End of Report --
"""

print("Sample incident report loaded for analysis!")

## Executive Summary Generation

Generate a high-level summary suitable for executives and business stakeholders.

In [ ]:
executive_system_prompt = "You are a SOC assistant summarizing a lengthy incident report for executive leadership."

print("=== EXECUTIVE SUMMARY ===")
summary_exec = inference(make_prompt(long_incident_doc, "executive"), executive_system_prompt)
display(Markdown(summary_exec))

## Tactical Summary Generation

Generate a technical summary with actionable details for security analysts.

In [ ]:
tactical_system_prompt = "You are a SOC assistant extracting key technical findings from a lengthy incident report for security analysts."

print("=== TACTICAL SUMMARY ===")
summary_tactical = inference(make_prompt(long_incident_doc, "tactical"), tactical_system_prompt)
display(Markdown(summary_tactical))

## Advanced Summarization Features

Let's create additional summarization capabilities for different use cases.

In [ ]:
def generate_ioc_summary(incident_document):
    """Extract and summarize Indicators of Compromise (IOCs)"""
    
    prompt = f"""
    Extract all Indicators of Compromise (IOCs) from this incident report and organize them by type:
    
    - IP Addresses
    - URLs/Domains
    - File Hashes
    - Hostnames/Systems
    - User Accounts
    - Scheduled Tasks/Services
    - Network Artifacts
    
    Include context for each IOC when available.
    
    ## INCIDENT REPORT
    {incident_document}
    """
    
    return inference(prompt, "You are a threat intelligence analyst extracting IOCs from incident reports.")

def generate_timeline_summary(incident_document):
    """Create a condensed timeline of key events"""
    
    prompt = f"""
    Create a condensed timeline focusing on the most critical events from this incident.
    Include:
    - Initial compromise
    - Key lateral movement events
    - Data access/exfiltration attempts
    - Detection and response actions
    
    Format as a bullet-point timeline with timestamps.
    
    ## INCIDENT REPORT
    {incident_document}
    """
    
    return inference(prompt, "You are a SOC analyst creating incident timelines.")

def generate_lessons_learned_summary(incident_document):
    """Extract and expand on lessons learned and recommendations"""
    
    prompt = f"""
    Analyze this incident report and provide:
    
    1. Key security gaps that enabled the attack
    2. Detection opportunities that were missed
    3. Response improvements for future incidents
    4. Prioritized recommendations for prevention
    
    Focus on actionable insights that can improve security posture.
    
    ## INCIDENT REPORT
    {incident_document}
    """
    
    return inference(prompt, "You are a security consultant analyzing incidents for improvement opportunities.")

print("Advanced summarization functions ready!")

In [ ]:
print("=== IOC SUMMARY ===")
ioc_summary = generate_ioc_summary(long_incident_doc)
display(Markdown(ioc_summary))

print("\n" + "="*60 + "\n")

print("=== TIMELINE SUMMARY ===")
timeline_summary = generate_timeline_summary(long_incident_doc)
display(Markdown(timeline_summary))

print("\n" + "="*60 + "\n")

print("=== LESSONS LEARNED SUMMARY ===")
lessons_summary = generate_lessons_learned_summary(long_incident_doc)
display(Markdown(lessons_summary))

## Multi-Format Report Generator

Create a comprehensive summarization tool that generates multiple output formats.

In [ ]:
def comprehensive_incident_summary(incident_document):
    """Generate a comprehensive multi-section summary"""
    
    print("Generating comprehensive incident summary...")
    
    # Generate all summary types
    executive = inference(make_prompt(incident_document, "executive"), 
                         "You are a SOC assistant summarizing for executives.")
    
    tactical = inference(make_prompt(incident_document, "tactical"), 
                        "You are a SOC assistant summarizing for analysts.")
    
    iocs = generate_ioc_summary(incident_document)
    timeline = generate_timeline_summary(incident_document)
    lessons = generate_lessons_learned_summary(incident_document)
    
    # Compile comprehensive report
    report = f"""
# Incident Summary Report

## Executive Summary
{executive}

## Tactical Summary
{tactical}

## Key Timeline
{timeline}

## Indicators of Compromise (IOCs)
{iocs}

## Lessons Learned & Recommendations
{lessons}

---
*Report generated automatically from incident documentation*
"""
    
    return report

print("=== COMPREHENSIVE INCIDENT SUMMARY ===")
comprehensive_report = comprehensive_incident_summary(long_incident_doc)
display(Markdown(comprehensive_report))

## Custom Incident Summarization

Analyze your own incident reports:

In [ ]:
# Add your custom incident report here
custom_incident_report = """
# Paste your incident report content here
# The system will generate executive and tactical summaries
"""

if custom_incident_report.strip() and not custom_incident_report.startswith("# Paste your"):
    print("=== ANALYZING CUSTOM INCIDENT REPORT ===")
    
    # Generate executive summary
    custom_exec = inference(make_prompt(custom_incident_report, "executive"), 
                           "You are a SOC assistant summarizing for executives.")
    
    print("\n--- Executive Summary ---")
    display(Markdown(custom_exec))
    
    # Generate tactical summary
    custom_tactical = inference(make_prompt(custom_incident_report, "tactical"), 
                               "You are a SOC assistant summarizing for analysts.")
    
    print("\n--- Tactical Summary ---")
    display(Markdown(custom_tactical))
    
    # Generate IOC summary
    custom_iocs = generate_ioc_summary(custom_incident_report)
    
    print("\n--- IOCs Summary ---")
    display(Markdown(custom_iocs))
    
else:
    print("Add your incident report content in the cell above to analyze it!")

## Summarization Best Practices

Tips for effective incident summarization:

In [ ]:
best_practices = """
# Incident Summarization Best Practices

## Executive Summaries
- **Focus on business impact**: What was affected and how?
- **Timeline significance**: When did it happen and how long did it last?
- **Resolution status**: What has been done to fix it?
- **Future prevention**: What will prevent recurrence?
- **Language**: Non-technical, business-focused

## Tactical Summaries
- **Technical details**: TTPs, IOCs, and attack progression
- **MITRE ATT&CK mapping**: Specific techniques used
- **Response actions**: What was done and why
- **Detection gaps**: What was missed and how to improve
- **Language**: Technical, analyst-focused

## Key Elements to Include
1. **Initial compromise vector**
2. **Lateral movement techniques**
3. **Data accessed or exfiltrated**
4. **Detection and response timeline**
5. **Lessons learned and recommendations**

## Common Pitfalls to Avoid
- Too much technical jargon in executive summaries
- Missing business impact in tactical summaries
- Incomplete IOC extraction
- Vague or generic recommendations
- Inconsistent timeline information
"""

display(Markdown(best_practices))

## Incident Summarization Summary

This notebook demonstrates automated incident summarization capabilities:

- **Executive Summaries**: High-level business-focused summaries for leadership
- **Tactical Summaries**: Technical details and IOCs for security analysts
- **Timeline Extraction**: Key events in chronological order
- **IOC Analysis**: Structured extraction of indicators of compromise
- **Lessons Learned**: Actionable insights for security improvement

These automated summaries significantly reduce the time needed to digest complex incident reports while ensuring consistent, comprehensive coverage of critical information for different stakeholder audiences.